# Meteorology-SnowDegreeDay coupling

**Goal:** Try to successfully run a coupled `Meteorology-SnowDegreeDay` simulation. Each component runs to completion in stand-alone mode.

Import the Babel-wrapped `Meteorology` and `SnowDegreeDay` components and create instances:

In [ ]:
from cmt.components import Meteorology, SnowDegreeDay
met, sno = Meteorology(), SnowDegreeDay()

## Demonstrate that the models run independently

Initialize the components with cfg files that, for simplicity, use the same time step and run duration:

In [ ]:
met.initialize('./input/meteorology-2.cfg')
sno.initialize('./input/snow_degree_day-1.cfg')

Get the model time step (which has been broadcast over the grid):

In [ ]:
time_step = met.get_value('model__time_step').max()
print 'Time step = {} s'.format(time_step)

Advance both models to the end, saving the model time, precipitation flux, and snowmelt flux values at each step:

In [ ]:
time, precip, melt = [], [], []
until = time_step
while met.get_current_time() < met.get_end_time():
    met.update(until)
    time.append(met.get_current_time())
    precip.append(met.get_value('atmosphere_water__precipitation_leq-volume_flux'))
    melt.append(sno.get_value('snowpack__melt_volume_flux'))
    until += time_step

Check the values:

In [ ]:
time

In [ ]:
precip

In [ ]:
melt

Finalize the components:

In [ ]:
met.finalize(), sno.finalize()

**Result:** The models run independently.

## Couple the models

Initialize the components with cfg files that, for simplicity, use the same time step and run duration:

In [ ]:
met.initialize('./input/meteorology-2.cfg')
sno.initialize('./input/snow_degree_day-1.cfg')

Get the model time step (which has been broadcast over the grid):

In [ ]:
time_step = met.get_value('model__time_step').max()
print 'Time step = {} s'.format(time_step)

Get the shape of the grid. See **Note** below.

In [ ]:
grid_id = met.get_var_grid('snowpack__melt_volume_flux')
shape = met.get_grid_shape(grid_id)
print 'Grid shape = {}'.format(shape)

Store initial values of time and net longwave radiative flux:

In [ ]:
time = [met.get_current_time()]
Qn_LW = [met.get_value('land_surface_net-longwave-radiation__energy_flux').max()]

Run the coupled models to completion. In each time step, perform the following actions:

1. Advance `SnowDegreeDay`
1. Get variables from `SnowDegreeDay`
1. Pass variables to `Meteorology`
1. Advance `Meteorology`

In [ ]:
until = time_step
while met.get_current_time() < met.get_end_time():
    sno.update(until)
    rho_snow = sno.get_value('snowpack__z_mean_of_mass-per-volume_density')
    h_snow = sno.get_value('snowpack__depth')
    h_swe = sno.get_value('snowpack__liquid-equivalent_depth')
    SM = sno.get_value('snowpack__melt_volume_flux')

    met.set_value('snowpack__z_mean_of_mass-per-volume_density', rho_snow.reshape(shape))
    met.set_value('snowpack__depth', h_snow.reshape(shape))
    met.set_value('snowpack__liquid-equivalent_depth', h_swe.reshape(shape))
    met.set_value('snowpack__melt_volume_flux', SM.reshape(shape))
    met.update(until)
    
    time.append(met.get_current_time())
    Qn_LW.append(met.get_value('land_surface_net-longwave-radiation__energy_flux').max())
    
    until += time_step

In [ ]:
print time

In [ ]:
print Qn_LW

Finalize the components:

In [ ]:
met.finalize(), sno.finalize()

**Note:** The Meteorology component was able to advance because I explicitly reshaped the variables `rho_snow`, `h_snow`, `h_swe`, and `SM` (which had been flattened) before passing them into `set_value`.

**Result:** Indeterminate. It depends whether my fix is robust.